In [1]:
import os

# Read the file and load the keys
token_file = open('token.txt', 'r')
# Read the first line
open_ai_token_line = token_file.readline()

if open_ai_token_line:
    open_ai_key = open_ai_token_line.split('=')[1].strip()
    print(f'Open AI Key  is loaded')

# Read the second line
langsmith_token_line = token_file.readline()

if langsmith_token_line:
    langsmith_key = langsmith_token_line.split('=')[1].strip()
    print(f'Langsmith Key is loaded')


if langsmith_key is not None:
    # Configure LangSmith
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = "MULTI-AGENT"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_API_KEY"] = langsmith_key

Open AI Key  is loaded
Langsmith Key is loaded


### Create agents

In [12]:
import json

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation


def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    functions = [convert_to_openai_function(t) for t in tools]

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards defining the key elements to create a View, according to given task. \
                    The final response should be a JSON text that includes filters and the join rules."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_functions(functions)

### Define tools

In [21]:
from langchain_core.tools import tool
from typing import Annotated
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.agents import Tool
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=open_ai_key)

# Read the metamodels files and create retrievel tools for each
meta1_file = open('temp/Book.txt', 'r')
meta2_file = open('temp/Publication.txt', 'r')

documents = [{"name": "Book", "content": meta1_file.read()}, {"name": "Publication", "content": meta2_file.read()}]
# Split documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=20, chunk_overlap=5)
texts = text_splitter.create_documents(list(d['content'] for d in documents))
# Select embeddings
embeddings = OpenAIEmbeddings(openai_api_key=open_ai_key)
# Create a vectorstore from documents
db = Chroma.from_documents(texts, embeddings)
# Create retriever interface
retriever = db.as_retriever()


tools = []

# Create QA chain inside the tools
tools.append(
    Tool(
        name=documents[0]["name"],
        description=f"useful when you want to answer questions about {documents[0]['name']}",
        func=RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever),
    )
)
tools.append(
    Tool(
        name=documents[1]["name"],
        description=f"useful when you want to answer questions about {documents[1]['name']}",
        func=RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever),
    )
)


#### Agent State

In [22]:
import operator
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union

from langchain.agents import create_openai_functions_agent
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from typing_extensions import TypedDict


# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

#### Agent Nodes

In [23]:
import functools

# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, FunctionMessage):
        pass
    else:
        result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }


# MetaclassAnalyzer agent and node
get_join_rules_agent = create_agent(
    llm, 
    [tools[0], tools[1]], 
    system_message="You should provide which elements can be combined in the final View, according to the given task. \
                    In a View, the elements are combined in pairs.\
                    Combining two elements means that the View will include a single element representing the same domain object \
                    Your answer should be a list of elements.\
                    Each element of the list is a dictionary containing the name of this virtual relation and a tuple with the combined elements in the following format:\
                          {{Relation_name: (Metamodel_Identifier.Class_name, Metamodel_Identifier.Class_name)}}\
                    Only use class names that actually exist in the metamodels; \
                        don't try to invent new class names. The relation's name should combine these class names, always in camelCase."
)
get_join_rules_node = functools.partial(agent_node, agent=get_join_rules_agent, name="GetJoinRules")

# Filter Generator
filter_generator_agent = create_agent(
    llm,
    [tools[0], tools[1]],
    system_message="You should provide which elements should be selected to be present in the final View.\
                Your answer should be a list of elements.\
                Each element is in the following format: Metamodel_Identifier.Class_name.\
                Only use class and attribute names that actually exist in the metamodels; don't try to invent new names.\
                Note that frequently, the metamodels can represent the same domain, so it's possible to get some overlap between them.\
                This should be taken into account to avoid repeating information."
)
filter_generator_node = functools.partial(agent_node, agent=filter_generator_agent, name="FilterGenerator")

#### Tool node

In [24]:
tools = [tools[0], tools[1]]
tool_executor = ToolExecutor(tools)

def tool_node(state):
    """This runs tools in the graph

    It takes in an agent action and calls that tool and returns the result."""
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    tool_input = json.loads(
        last_message.additional_kwargs["function_call"]["arguments"]
    )
    # We can pass single-arg inputs by value
    if len(tool_input) == 1 and "__arg1" in tool_input:
        tool_input = next(iter(tool_input.values()))
    tool_name = last_message.additional_kwargs["function_call"]["name"]
    action = ToolInvocation(
        tool=tool_name,
        tool_input=tool_input,
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(
        content=f"{tool_name} response: {str(response)}", name=action.tool
    )
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

#### Edge

In [25]:
# Either agent can decide to end
def router(state):
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "function_call" in last_message.additional_kwargs:
        # The previus agent is invoking a tool
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "end"
    return "continue"

In [26]:
workflow = StateGraph(AgentState)

workflow.add_node("GetJoinRules", get_join_rules_node)
workflow.add_node("FilterGenerator", filter_generator_node)
workflow.add_node("call_tool", tool_node)

workflow.add_conditional_edges(
    "GetJoinRules",
    router,
    {"continue": "FilterGenerator", "call_tool": "call_tool", "end": END},
)
workflow.add_conditional_edges(
    "FilterGenerator",
    router,
    {"continue": "GetJoinRules", "call_tool": "call_tool", "end": END},
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {
        "GetJoinRules": "GetJoinRules",
        "FilterGenerator": "FilterGenerator",
    },
)
workflow.set_entry_point("GetJoinRules")
graph = workflow.compile()

In [27]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Since the same book being represented by a Book model or by a Publication model, the task is to define the best way to combine a book with a publication getting all the information in the same View"
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 100},
):
    print(s)
    print("----")

{'GetJoinRules': {'messages': [HumanMessage(content='To combine a Book with a Publication and get all the information in the same View, we can create a virtual relation that includes both the Book and Publication models. Here is the combination:\n\n1. Combine Book and Publication:\n   - Relation name: bookPublication\n   - Combined elements: {bookPublication: ("Book", "Publication")}\n\nThis combination will allow us to access all the information related to a book, regardless of whether it is represented by a Book model or a Publication model.', name='GetJoinRules')], 'sender': 'GetJoinRules'}}
----
{'FilterGenerator': {'messages': [HumanMessage(content='To further clarify, here is the JSON representation of the combined relation:\n\n```json\n{\n  "relationName": "bookPublication",\n  "combinedElements": {\n    "bookPublication": [\n      "Book",\n      "Publication"\n    ]\n  }\n}\n```', name='FilterGenerator')], 'sender': 'FilterGenerator'}}
----
{'GetJoinRules': {'messages': [HumanM

JSONDecodeError: Expecting value: line 1 column 1 (char 0)